<a href="https://colab.research.google.com/github/davidlealo/100profes/blob/master/Capturar_audio_del_microfono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from IPython.display import Javascript, display

def create_buttons(filename="grabacion.wav"):
    # Escapar el nombre del archivo en caso de caracteres especiales
    filename_escaped = filename.replace("'", "\\'").replace('"', '\\"')

    display(Javascript(f"""
        // Crear los elementos de la interfaz
        var startButton = document.createElement('button');
        startButton.innerHTML = 'Comenzar Grabación';
        startButton.style.marginRight = '10px';

        var stopButton = document.createElement('button');
        stopButton.innerHTML = 'Detener Grabación';
        stopButton.disabled = true;

        var statusText = document.createElement('p');
        statusText.innerHTML = '<strong>Status:</strong> Listo para grabar';

        document.body.appendChild(startButton);
        document.body.appendChild(stopButton);
        document.body.appendChild(statusText);

        var audioChunks = [];
        var mediaRecorder;

        startButton.onclick = async function() {{
            // Solicitar acceso al micrófono
            const stream = await navigator.mediaDevices.getUserMedia({{ audio: true }});
            mediaRecorder = new MediaRecorder(stream);

            mediaRecorder.ondataavailable = function(event) {{
                audioChunks.push(event.data);
            }};

            mediaRecorder.onstop = function() {{
                // Crear el archivo de audio y el enlace de descarga
                const audioBlob = new Blob(audioChunks, {{ type: 'audio/wav' }});
                const audioUrl = URL.createObjectURL(audioBlob);
                const downloadLink = document.createElement('a');
                downloadLink.href = audioUrl;
                downloadLink.download = '{filename_escaped}';
                downloadLink.click();

                audioChunks = [];
            }};

            mediaRecorder.start();
            statusText.innerHTML = '<strong>Status:</strong> Grabando...';
            startButton.disabled = true;
            stopButton.disabled = false;
        }};

        stopButton.onclick = function() {{
            mediaRecorder.stop();
            statusText.innerHTML = '<strong>Status:</strong> Grabación detenida.';
            startButton.disabled = false;
            stopButton.disabled = true;
        }};
    """))

# Llama a la función para crear los botones
create_buttons("conversacion.wav")


<IPython.core.display.Javascript object>

In [6]:
!pip install pydub speechrecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 31.7 MB/s eta 0:00:00


In [7]:
from IPython.display import Javascript, display
from google.colab import files
import speech_recognition as sr
from pydub import AudioSegment

# Crear el botón de carga usando JavaScript
def upload_audio_button():
    display(Javascript('''
        async function uploadAudio() {
            // Crear un botón en la interfaz
            const fileInput = document.createElement('input');
            fileInput.type = 'file';
            fileInput.accept = '.mp3';
            fileInput.onchange = async () => {
                // Obtener el archivo seleccionado
                const file = fileInput.files[0];

                // Subir el archivo a Colab
                const data = new FormData();
                data.append('file', file);

                // Llamada a la API de Colab para cargar el archivo
                const response = await fetch('/upload', { method: 'POST', body: data });
                if (response.ok) {
                    google.colab.output.resume();
                    console.log('Archivo subido con éxito:', file.name);
                } else {
                    console.error('Error al subir el archivo.');
                }
            };

            fileInput.click();
        }

        // Crear y mostrar el botón
        const button = document.createElement('button');
        button.innerText = 'Subir y procesar archivo de audio';
        button.onclick = uploadAudio;
        document.body.appendChild(button);
    '''))

# Llama a la función para mostrar el botón
upload_audio_button()


<IPython.core.display.Javascript object>

In [8]:
# Define el nombre del archivo .mp3 que subiste
input_file = 'nombre_del_archivo.mp3'  # Cambia este nombre al de tu archivo subido
output_file = 'archivo_convertido.wav'

# Convertir el archivo de audio a formato .wav
audio = AudioSegment.from_mp3(input_file)
audio.export(output_file, format="wav")

# Configura el reconocimiento de voz
recognizer = sr.Recognizer()

# Abre el archivo de audio y usa speech_recognition para transcribir
with sr.AudioFile(output_file) as source:
    audio_data = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio_data, language='es-ES')  # Cambia el idioma si es necesario
        print("Transcripción del audio:", text)
    except sr.UnknownValueError:
        print("No se pudo entender el audio.")
    except sr.RequestError as e:
        print(f"Error al solicitar los resultados de Google Speech Recognition; {e}")


FileNotFoundError: [Errno 2] No such file or directory: 'nombre_del_archivo.mp3'